In [1]:
import findspark
findspark.init()

import pyspark

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import col, expr, translate,  trim, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, NGram
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

import os
print(os.environ.get('SPARK_HOME'))
print(os.environ.get('PYSPARK_HOME'))
print(os.environ.get('PYTHONPATH'))

/usr/local/spark
/home/ubuntu/.local/bin/pyspark
/usr/local/spark/python:/home/ubuntu/.local/bin/pyspark/python:


In [13]:
import sys
import argparse
import unicodedata
from pyspark import SparkContext,SparkConf,SQLContext
from pyspark.sql import Row,SparkSession,HiveContext
from pyspark.sql.functions import col,size,explode,split
from pyspark.sql.types import StringType,IntegerType,ArrayType
from pyspark.sql.functions import udf, array, length
from bs4 import BeautifulSoup
import mistune
import re
from nltk.corpus import stopwords


ModuleNotFoundError: No module named 'nltk'

In [18]:
print(os.environ.get('SPARK_HOME'))  
print(pyspark.__file__) 

/usr/local/spark
/usr/local/spark/python/pyspark/__init__.py


In [9]:
conf = SparkConf() \
    .setAppName("Group1_2 Project") \
    .set("spark.hadoop.dfs.namenode", "hdfs://grouop2master:9866") \
    .set("spark.sql.warehouse.dir", "hdfs://grouop2master:9866/user/hive/warehouse")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [19]:
if spark is not None:
    print(spark.sparkContext.appName)  # Print the application name if it exists
else:
    print("No active SparkContext")

Group1_2 Project


In [22]:
hdfs_path = "hdfs://grouop2master:9000//user/ubuntu/corpus-webis-tldr-17.json" 
df = spark.read.json(hdfs_path)
df.show()

+------------------+--------------------+--------------------+-----------+-------+--------------------+--------------------+------------+--------------------+-----------+--------+
|            author|                body|             content|content_len|     id|      normalizedBody|           subreddit|subreddit_id|             summary|summary_len|   title|
+------------------+--------------------+--------------------+-----------+-------+--------------------+--------------------+------------+--------------------+-----------+--------+
|  raysofdarkmatter|I think it should...|I think it should...|        178|c69al3r|I think it should...|                math|    t5_2qh0n|Shifting seasonal...|          8|    null|
|           Stork13|Art is about the ...|Art is about the ...|        148|c6a9nxd|Art is about the ...|               funny|    t5_2qh33|Personal opinions...|          4|    null|
|     Cloud_dreamer|Ask me what I thi...|Ask me what I thi...|         76|c6acx4l|Ask me what I thi.

In [24]:
df = df.select("body", "subreddit")  
df.show()

+--------------------+--------------------+
|                body|           subreddit|
+--------------------+--------------------+
|I think it should...|                math|
|Art is about the ...|               funny|
|Ask me what I thi...|         Borderlands|
|In Mechwarrior On...|            gamingpc|
|You are talking a...|              Diablo|
|All but one of my...|   RedditLaqueristas|
|I could give a sh...|               apple|
|So you're saying ...|               apple|
|I love this idea ...|RedditFilmsProduc...|
|Theres an entire ...|       AbandonedPorn|
|FALSE. Evidence: ...|             atheism|
|If the number of ...|              quotes|
|Yeah, but most fo...|           AskReddit|
|As an entrepreneu...|     personalfinance|
|i guess the way I...|             Animals|
|Didn't they lose ...|     leagueoflegends|
|You probably won'...|           AskReddit|
|To simply say tha...|              videos|
|This picture does...|               trees|
|And that is, hand...|          

In [25]:
df = df.withColumn("subreddit", lower(df["subreddit"]))# convert subreddit categories to lowercase
df = df.withColumn("subreddit", trim("subreddit"))# remove white spaces in subreddit
df.show()

+--------------------+--------------------+
|                body|           subreddit|
+--------------------+--------------------+
|I think it should...|                math|
|Art is about the ...|               funny|
|Ask me what I thi...|         borderlands|
|In Mechwarrior On...|            gamingpc|
|You are talking a...|              diablo|
|All but one of my...|   redditlaqueristas|
|I could give a sh...|               apple|
|So you're saying ...|               apple|
|I love this idea ...|redditfilmsproduc...|
|Theres an entire ...|       abandonedporn|
|FALSE. Evidence: ...|             atheism|
|If the number of ...|              quotes|
|Yeah, but most fo...|           askreddit|
|As an entrepreneu...|     personalfinance|
|i guess the way I...|             animals|
|Didn't they lose ...|     leagueoflegends|
|You probably won'...|           askreddit|
|To simply say tha...|              videos|
|This picture does...|               trees|
|And that is, hand...|          

In [26]:
df = df.withColumn('body', translate('body', '1234567890!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~', ' '))# remove punctuation and numbers in the body column
df.show()

+--------------------+--------------------+
|                body|           subreddit|
+--------------------+--------------------+
|I think it should...|                math|
|Art is about the ...|               funny|
|Ask me what I thi...|         borderlands|
|In Mechwarrior On...|            gamingpc|
|You are talking a...|              diablo|
|All but one of my...|   redditlaqueristas|
|I could give a sh...|               apple|
|So youre saying t...|               apple|
|I love this idea ...|redditfilmsproduc...|
|Theres an entire ...|       abandonedporn|
|FALSE Evidence Wo...|             atheism|
|If the number of ...|              quotes|
|Yeah but most fol...|           askreddit|
|As an entrepreneu...|     personalfinance|
|i guess the way I...|             animals|
|Didnt they lose  ...|     leagueoflegends|
|You probably wont...|           askreddit|
|To simply say tha...|              videos|
|This picture does...|               trees|
|And that is hands...|          

In [27]:
df = df.withColumn("body", regexp_replace(col("body"), "\\s+", " "))#remove white spaces in body
df.show()


+--------------------+--------------------+
|                body|           subreddit|
+--------------------+--------------------+
|I think it should...|                math|
|Art is about the ...|               funny|
|Ask me what I thi...|         borderlands|
|In Mechwarrior On...|            gamingpc|
|You are talking a...|              diablo|
|All but one of my...|   redditlaqueristas|
|I could give a sh...|               apple|
|So youre saying t...|               apple|
|I love this idea ...|redditfilmsproduc...|
|Theres an entire ...|       abandonedporn|
|FALSE Evidence Wo...|             atheism|
|If the number of ...|              quotes|
|Yeah but most fol...|           askreddit|
|As an entrepreneu...|     personalfinance|
|i guess the way I...|             animals|
|Didnt they lose g...|     leagueoflegends|
|You probably wont...|           askreddit|
|To simply say tha...|              videos|
|This picture does...|               trees|
|And that is hands...|          

In [28]:
df = df.filter(df.body != '')# delete empty lines
df.show()


+--------------------+--------------------+
|                body|           subreddit|
+--------------------+--------------------+
|I think it should...|                math|
|Art is about the ...|               funny|
|Ask me what I thi...|         borderlands|
|In Mechwarrior On...|            gamingpc|
|You are talking a...|              diablo|
|All but one of my...|   redditlaqueristas|
|I could give a sh...|               apple|
|So youre saying t...|               apple|
|I love this idea ...|redditfilmsproduc...|
|Theres an entire ...|       abandonedporn|
|FALSE Evidence Wo...|             atheism|
|If the number of ...|              quotes|
|Yeah but most fol...|           askreddit|
|As an entrepreneu...|     personalfinance|
|i guess the way I...|             animals|
|Didnt they lose g...|     leagueoflegends|
|You probably wont...|           askreddit|
|To simply say tha...|              videos|
|This picture does...|               trees|
|And that is hands...|          

In [29]:
tokenizer = Tokenizer(inputCol="body", outputCol="words")
df = tokenizer.transform(df)# split each sentence into separate words
df.show()

+--------------------+--------------------+--------------------+
|                body|           subreddit|               words|
+--------------------+--------------------+--------------------+
|I think it should...|                math|[i, think, it, sh...|
|Art is about the ...|               funny|[art, is, about, ...|
|Ask me what I thi...|         borderlands|[ask, me, what, i...|
|In Mechwarrior On...|            gamingpc|[in, mechwarrior,...|
|You are talking a...|              diablo|[you, are, talkin...|
|All but one of my...|   redditlaqueristas|[all, but, one, o...|
|I could give a sh...|               apple|[i, could, give, ...|
|So youre saying t...|               apple|[so, youre, sayin...|
|I love this idea ...|redditfilmsproduc...|[i, love, this, i...|
|Theres an entire ...|       abandonedporn|[theres, an, enti...|
|FALSE Evidence Wo...|             atheism|[false, evidence,...|
|If the number of ...|              quotes|[if, the, number,...|
|Yeah but most fol...|   

In [30]:
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
df = stopwords_remover.transform(df) # remove stopwords
df.show()

+--------------------+--------------------+--------------------+--------------------+
|                body|           subreddit|               words|      filtered_words|
+--------------------+--------------------+--------------------+--------------------+
|I think it should...|                math|[i, think, it, sh...|[think, fixed, ei...|
|Art is about the ...|               funny|[art, is, about, ...|[art, hardest, th...|
|Ask me what I thi...|         borderlands|[ask, me, what, i...|[ask, think, wall...|
|In Mechwarrior On...|            gamingpc|[in, mechwarrior,...|[mechwarrior, onl...|
|You are talking a...|              diablo|[you, are, talkin...|[talking, charsi,...|
|All but one of my...|   redditlaqueristas|[all, but, one, o...|[one, nails, ball...|
|I could give a sh...|               apple|[i, could, give, ...|[give, shit, turn...|
|So youre saying t...|               apple|[so, youre, sayin...|[youre, saying, t...|
|I love this idea ...|redditfilmsproduc...|[i, love, t

In [34]:
result = df.groupBy("subreddit").agg(
    F.sum(F.size("words")).alias("total_word_count"),#calculate the number of all words in each subreddit category
    F.sum(F.size("filtered_words")).alias("other_word_count"),#calculate the number of words after removing stop words in each subreddit category
    F.count("body").alias("sentence_count")#calculate the number of requests in each subreddit category
)

result.show()



+--------------------+----------------+----------------+--------------+
|           subreddit|total_word_count|other_word_count|sentence_count|
+--------------------+----------------+----------------+--------------+
|               anime|         1565594|          806284|          5868|
|              travel|          885554|          447319|          3076|
|londonfootballmeetup|            1802|             888|             6|
|     youtubecomments|            1303|             698|             8|
|                cubs|           21827|           11318|            85|
|              poetry|           64591|           32451|           220|
|                tmnt|           20841|           10652|            54|
|      dippingtobacco|           53285|           26489|           289|
|       crohnsdisease|          193440|           95665|           594|
|           metro2033|            1967|            1068|            16|
|             marxism|            1822|             958|        

In [32]:
popular_subreddit= result.orderBy(col("sentence_count").desc())# find the most popular category
popular_subreddit.show(truncate=False)

+-------------------+----------------+----------------+--------------+
|subreddit          |total_word_count|other_word_count|sentence_count|
+-------------------+----------------+----------------+--------------+
|askreddit          |145463239       |70295937        |589947        |
|relationships      |173638495       |78013133        |352049        |
|leagueoflegends    |25763882        |13366879        |109307        |
|tifu               |19268244        |9257272         |52219         |
|relationship_advice|24645327        |11050084        |50416         |
|trees              |11815990        |5747224         |47286         |
|gaming             |9176368         |4696013         |43851         |
|atheism            |11621663        |5666301         |43268         |
|adviceanimals      |7872130         |3883801         |40783         |
|funny              |6947389         |3497120         |40171         |
|politics           |8614353         |4429484         |36518         |
|pics 

In [35]:
most_popular_subredits = result.filter(result.sentence_count > 10)# display the category, where the percentage of stopwords is the highest and the number of requests is more than 10
most_popular_subredits.orderBy(col("stop_word_percentage").desc()).show()

AnalysisException: cannot resolve '`stop_word_percentage`' given input columns: [other_word_count, sentence_count, subreddit, total_word_count];;
'Sort ['stop_word_percentage DESC NULLS LAST], true
+- Filter (sentence_count#268L > cast(10 as bigint))
   +- Aggregate [subreddit#101], [subreddit#101, sum(cast(size(words#156, true) as bigint)) AS total_word_count#264L, sum(cast(size(filtered_words#177, true) as bigint)) AS other_word_count#266L, count(body#129) AS sentence_count#268L]
      +- Project [body#129, subreddit#101, words#156, UDF(words#156) AS filtered_words#177]
         +- Project [body#129, subreddit#101, UDF(body#129) AS words#156]
            +- Filter NOT (body#129 = )
               +- Project [regexp_replace(body#115, \s+,  ) AS body#129, subreddit#101]
                  +- Project [translate(body#8, 1234567890!"#$%&'()*+,-./:;<=>?@[\]^_{|}~,  ) AS body#115, subreddit#101]
                     +- Project [body#8, trim(subreddit#98, None) AS subreddit#101]
             

In [36]:
ngram = NGram(n=2, inputCol="words", outputCol="ngrams2")# ngram calculation where n=2
df = ngram.transform(df)
df.select("ngrams2").show()


+--------------------+
|             ngrams2|
+--------------------+
|[i think, think i...|
|[art is, is about...|
|[ask me, me what,...|
|[in mechwarrior, ...|
|[you are, are tal...|
|[all but, but one...|
|[i could, could g...|
|[so youre, youre ...|
|[i love, love thi...|
|[theres an, an en...|
|[false evidence, ...|
|[if the, the numb...|
|[yeah but, but mo...|
|[as an, an entrep...|
|[i guess, guess t...|
|[didnt they, they...|
|[you probably, pr...|
|[to simply, simpl...|
|[this picture, pi...|
|[and that, that i...|
+--------------------+
only showing top 20 rows



In [37]:
all_pairs_df = df.select(F.explode("ngrams2").alias("pair2"))# transform each pair into a separate row
pair_counts = all_pairs_df.groupBy("pair2").count()# count the number of pairs
most_common_word_pair = pair_counts.orderBy(F.desc("count")).first()
print(most_common_word_pair)# print the most popular pair
pair_counts.orderBy(F.desc("count")).show()# print first 20 popular pairs

Row(pair2='of the', count=3478162)


+-------+-------+
|  pair2|  count|
+-------+-------+
| of the|3478162|
| in the|3389036|
|  and i|2871662|
|  i was|2865442|
|  to be|2200242|
| to the|2013867|
| i have|1898029|
| on the|1744995|
| that i|1665249|
| i dont|1658424|
|  but i|1568086|
| it was|1517470|
|want to|1502303|
|   i am|1461029|
|for the|1420812|
|   in a|1403413|
|  for a|1313219|
|   so i|1284937|
| to get|1263131|
| if you|1158753|
+-------+-------+
only showing top 20 rows



In [38]:
ngram = NGram(n=3, inputCol="words", outputCol="ngrams")# ngram calculation where n=3
df = ngram.transform(df)
df.select("ngrams").show()


+--------------------+
|              ngrams|
+--------------------+
|[i think it, thin...|
|[art is about, is...|
|[ask me what, me ...|
|[in mechwarrior o...|
|[you are talking,...|
|[all but one, but...|
|[i could give, co...|
|[so youre saying,...|
|[i love this, lov...|
|[theres an entire...|
|[false evidence w...|
|[if the number, t...|
|[yeah but most, b...|
|[as an entreprene...|
|[i guess the, gue...|
|[didnt they lose,...|
|[you probably won...|
|[to simply say, s...|
|[this picture doe...|
|[and that is, tha...|
+--------------------+
only showing top 20 rows



In [ ]:
all_pairs_df = df.select(F.explode("ngrams").alias("pair"))# transform each pair into a separate row
pair_counts = all_pairs_df.groupBy("pair").count()# count the number of pairs
most_common_word_pair = pair_counts.orderBy(F.desc("count")).first()
print(most_common_word_pair)# print the most popular pair
pair_counts.orderBy(F.desc("count")).show()# print first 20 popular pairs